In [1]:
import pyspark
from datetime import datetime
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import udf,lit

from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [2]:
d = sqlContext.read.format('com.databricks.spark.csv').options(inferschema='true')\
    .load('../BDdata/airports.csv')
d.where(d._c4 == "ILE").show()

+---+---+---+---+---+---+---+---+---+---+----+----+----+----+
|_c0|_c1|_c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|
+---+---+---+---+---+---+---+---+---+---+----+----+----+----+
+---+---+---+---+---+---+---+---+---+---+----+----+----+----+



In [3]:
d1 = d.select("_c4","_c6","_c7")
d1.where(d._c4 == "ILE").show()

+---+---+---+
|_c4|_c6|_c7|
+---+---+---+
+---+---+---+



In [4]:
res = sqlContext.read.format('com.databricks.spark.csv').options(inferschema='true',header='true')\
    .load('./results/query4.csv/part-00000-b110f8a4-bc28-401c-a889-da5ce0c4b6b0-c000.csv')
res.show()

+----+----+---------------+--------------+-------+-------+
|Year|Week|OutgoingPenalty|IngoingPenalty|Airport|Penalty|
+----+----+---------------+--------------+-------+-------+
|2000|  35|            0.0|           0.0|    STX|    0.0|
|1994|   3|            0.0|           0.0|    GJT|    0.0|
|2008|  16|            0.0|           0.0|    PSE|    0.0|
|2008|  45|            0.0|           0.0|    GTR|    0.0|
|1996|  22|            0.0|           0.0|    KOA|    0.0|
|1999|   2|            0.0|           0.0|    KOA|    0.0|
|1994|   8|            0.0|           0.0|    BET|    0.0|
|2002|   3|            0.0|           0.0|    CAK|    0.0|
|1998|  37|            0.0|           0.0|    LIH|    0.0|
|1997|   5|            0.0|           0.0|    GUC|    0.0|
|2002|  14|            0.0|           0.0|    DRO|    0.0|
|2003|  40|            0.0|           0.0|    LIH|    0.0|
|2003|  35|            0.0|           0.0|    MOT|    0.0|
|2003|  31|            0.0|           0.0|    OXR|    0.

In [6]:
joined = res.join(d1, on=[res.Airport == d1._c4]).drop("_c4").withColumnRenamed("_c6","lat").withColumnRenamed("_c7","lon")
joined.show()

+----+----+---------------+--------------+-------+-------+------------------+-------------------+
|Year|Week|OutgoingPenalty|IngoingPenalty|Airport|Penalty|               lat|                lon|
+----+----+---------------+--------------+-------+-------+------------------+-------------------+
|1994|   3|            0.0|           0.0|    GJT|    0.0|     39.1223983765|     -108.527000427|
|2008|  45|            0.0|           0.0|    GTR|    0.0|33.450298309299995| -88.59140014649999|
|1996|  22|            0.0|           0.0|    KOA|    0.0|19.738800048828125|-156.04600524902344|
|1999|   2|            0.0|           0.0|    KOA|    0.0|19.738800048828125|-156.04600524902344|
|1994|   8|            0.0|           0.0|    BET|    0.0|       60.77980042|       -161.8379974|
|2002|   3|            0.0|           0.0|    CAK|    0.0|40.916099548339844| -81.44219970703125|
|1998|  37|            0.0|           0.0|    LIH|    0.0| 21.97599983215332|-159.33900451660156|
|1997|   5|         

In [7]:
res.count() , joined.count()

(172882, 169951)

In [8]:
r = res.select("Airport").distinct().toLocalIterator()
j = joined.select("Airport").distinct().toLocalIterator()


In [9]:
set(r).difference(set(j))

{Row(Airport='BQN'),
 Row(Airport='ILE'),
 Row(Airport='MAZ'),
 Row(Airport='PSE'),
 Row(Airport='SJU'),
 Row(Airport='STT'),
 Row(Airport='STX')}

In [10]:
joined.coalesce(1).write.csv("results/query4.csv/res_with_lat_lon.csv",header=True)